## 步驟一、一次存一個月份的url

In [3]:
#把分類跟 url 組合在一起
import requests
from bs4 import BeautifulSoup
from urlparse import urljoin
import os
import datetime as dt
import time
#製造日期的迴圈
urls = ""
startdate = dt.datetime(2016, 4,1)
endate = dt.datetime(2016, 4,30)
totaldays = (endate - startdate).days + 1
for daynumber in range(totaldays):
    datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y%m%d")
    file_datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y%m")  # for filename 
    #print datestring
    #印出每一天的蘋果新聞 url
    page_url = 'http://www.appledaily.com.tw/appledaily/archive/{}'.format(datestring)
    print page_url  
    main_url ='http://www.appledaily.com.tw/appledaily'
    res = requests.get(page_url)
    soup = BeautifulSoup(res.text)
    #包含所有種類的div
    div = soup.select('.abdominis.clearmen')[0]
    #抓取所有連結
    #news_url=[]
    article = div.select('.nclns')
    #兩層迴圈，把類別跟 url 組合起來
    for i in article:
        
        category = i.select('h2')[0].text.encode("utf8")  
        for li in i.select('li'):
            href = li.select('a')[0]['href']
            news_url = urljoin(main_url,href).encode("utf8") +'\n' #urljoin會自動合併兩個網址
            urls += category + news_url  #先都存起來
    time.sleep(0.6)
    f = open('C:/Users/BIG DATA/Desktop/Apple_news/{}.text'.format(file_datestring),'w')
    f.write(urls) #用encode('utf-8)    #最後再一次寫出來
    f.close()    

http://www.appledaily.com.tw/appledaily/archive/20160401
http://www.appledaily.com.tw/appledaily/archive/20160402
http://www.appledaily.com.tw/appledaily/archive/20160403
http://www.appledaily.com.tw/appledaily/archive/20160404
http://www.appledaily.com.tw/appledaily/archive/20160405
http://www.appledaily.com.tw/appledaily/archive/20160406
http://www.appledaily.com.tw/appledaily/archive/20160407
http://www.appledaily.com.tw/appledaily/archive/20160408
http://www.appledaily.com.tw/appledaily/archive/20160409
http://www.appledaily.com.tw/appledaily/archive/20160410
http://www.appledaily.com.tw/appledaily/archive/20160411
http://www.appledaily.com.tw/appledaily/archive/20160412
http://www.appledaily.com.tw/appledaily/archive/20160413
http://www.appledaily.com.tw/appledaily/archive/20160414
http://www.appledaily.com.tw/appledaily/archive/20160415
http://www.appledaily.com.tw/appledaily/archive/20160416
http://www.appledaily.com.tw/appledaily/archive/20160417
http://www.appledaily.com.tw/ap

## 步驟二、讀取一個月的url檔案，並抓下來

In [8]:
import string
import datetime as dt
import requests
import time
from bs4 import BeautifulSoup
import json
import ast  #轉換成json需要套件
import time
count = 0
L = []
#製造日期的迴圈
startdate = dt.datetime(2016, 2,1)
endate = dt.datetime(2016, 2,2)
totaldays = (endate - startdate).days + 1
all_json_str = ''
for daynumber in range(totaldays):
    file_datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y%m")  # for filename 
    #開起每天的url檔案
    f = open('C:/Users/BIG DATA/Desktop/Apple_news/{}.text'.format(file_datestring),'r')
    for url_no in f.readlines():
        try:
            category = url_no.split('http')[0]        #把類別抓出來
            url = 'http'+url_no.split('http')[1]      #重新組合url
            res = requests.get(url)
            soup = BeautifulSoup(res.text)
            file_date,file_num = url.split('/')[6],url.split('/')[7]  #替檔案取名子    

            #抓標題跟副標題
            title1 = soup.select('#h1')[0].text
            title2 = soup.select('#h2')[0].text            
            title =''
            titles = str(title1.encode('utf-8'))+str(title2.encode('utf-8'))
            title = ','.join(titles.split())
            #print title
            
            #抓內文
            article = ''
            div= soup.select('.articulum')[0]   #先把所有內文抓到
            ps = div.select('p')                #再把裡面所有的p標籤抓到
            for p in ps:                        #把每個p標籤一個一個抓出來印
                article += p.text.replace('\r','')  #段落會隱藏換行符號\r，要取代掉，不然後面讀取json格式會只讀到最後一行

            #抓日期
            #date = soup.select('.gggs')[0].select('time')[0].text
            #f.write(time.encode('utf-8')+'\n')
        
            data  =  { 
                'comp':'apple',  #新聞來源
                'category':category,
                'title':title, 
                'content':article,
                'date':file_date,  #直接抓檔案的日期來用
                'url':url,
                'keyw':'',
                'hitcount':0
            }
             
            all_json_str += str(data)+','  #加個,把多本字典分開來
            #爬完一個網址休息一下
            time.sleep(0.6)
            
        #印出有問題的新聞
        except IndexError:
            count +=1
            print count
            continue
    #url 檔案關閉
    f.close()
    
f1 = open('C:/Users/BIG DATA/Desktop/Apple_news/json/{}.json'.format(file_datestring),'w')
#為了讓多本字典存到同一個檔案之中，會在外面產生[]
a = eval(all_json_str) #AST套件 把多本字典轉換成 list (eval是可剝離最外層包裝的函式)
#----------------------------------------------
b =json.dumps(a) #再把 list 字典轉換成 json 並包起來
f1.write(b)
f1.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


KeyboardInterrupt: 

In [4]:
#這個才是最正確的
#修改成一次爬好幾個月
import string
import datetime as dt
import requests
import time
from bs4 import BeautifulSoup
import json
import ast  #轉換成json需要套件
import time
count = 0
#製造日期的迴圈
for daynumber in range(4,5):
    all_json_str = []
    #開起每天的url檔案
    f = open('C:/Users/BIG DATA/Desktop/Apple_news/20160{}.text'.format(daynumber),'r')
    for url_no in f.readlines():
        try:
            category = url_no.split('http')[0]        #把類別抓出來
            url = 'http'+url_no.split('http')[1]      #重新組合url
            res = requests.get(url)
            soup = BeautifulSoup(res.text)
            file_date,file_num = url.split('/')[6],url.split('/')[7]  #替檔案取名子    

            #抓標題跟副標題
            title1 = soup.select('#h1')[0].text
            title2 = soup.select('#h2')[0].text            
            title =''
            titles = str(title1.encode('utf-8'))+str(title2.encode('utf-8'))
            title = ','.join(titles.split())
            #print title
            
            #抓內文
            article = ''
            div= soup.select('.articulum')[0]   #先把所有內文抓到
            ps = div.select('p')                #再把裡面所有的p標籤抓到
            for p in ps:                        #把每個p標籤一個一個抓出來印
                article += p.text.replace('\r','')  #段落會隱藏換行符號\r，要取代掉，不然後面讀取json格式會只讀到最後一行

            #抓日期
            #date = soup.select('.gggs')[0].select('time')[0].text
            #f.write(time.encode('utf-8')+'\n')
        
            data  =  { 
                'comp':'apple',  #新聞來源
                'category':category,
                'title':title, 
                'content':article,
                'date':file_date,  #直接抓檔案的日期來用
                'url':url,
                'keyw':'',
                'hitcount':0
            }
             
            all_json_str.append(data) #加個,把多本字典分開來
            #爬完一個網址休息一下
            time.sleep(1)
        #印出有問題的新聞
        except IndexError:
            count +=1
            print count
            continue
    #url 檔案關閉
    f1 = open('C:/Users/BIG DATA/Desktop/Apple_news/json/20160{}.json'.format(daynumber),'w')
    b =json.dumps(all_json_str) #再把 list 字典轉換成 json 並包起來
    f1.write(b)
    f1.close()
    f.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


IOError: [Errno 2] No such file or directory: 'C:/Users/BIG DATA/Desktop/Apple_news/json/201604.json'

In [6]:
f1 = open('C:/Users/BIG DATA/Desktop/Apple_news/json/20160{}.json'.format(daynumber),'w')
b =json.dumps(all_json_str) #再把 list 字典轉換成 json 並包起來
f1.write(b)
f1.close()
f.close()

## 步驟三、爬取每一天的url

In [168]:
#把分類跟 url 組合在一起
import requests
from bs4 import BeautifulSoup
from urlparse import urljoin
import os
import datetime as dt
import time
#製造日期的迴圈
urls = ""
startdate = dt.datetime(2016, 3,12)
endate = dt.datetime(2016, 3,12)
totaldays = (endate - startdate).days + 1
for daynumber in range(totaldays):
    datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y%m%d") 
    #print datestring
    
    #印出每一天的蘋果新聞 url
    page_url = 'http://www.appledaily.com.tw/appledaily/archive/{}'.format(datestring)
    print page_url  
    main_url ='http://www.appledaily.com.tw/appledaily'
    res = requests.get(page_url)
    soup = BeautifulSoup(res.text)
    #包含所有種類的div
    div = soup.select('.abdominis.clearmen')[0]
    #抓取所有連結
    #news_url=[]
    article = div.select('.nclns')
    
    #兩層迴圈，把類別跟 url 組合起來
    for i in article:
        f = open('C:/Users/BIG DATA/Desktop/Apple_news/{}.text'.format(datestring),'w')
        category = i.select('h2')[0].text.encode("utf8")  
        for li in i.select('li'):
            href = li.select('a')[0]['href']
            news_url = urljoin(main_url,href).encode("utf8") +'\n' #urljoin會自動合併兩個網址
            urls += category + news_url  #先都存起來
    time.sleep(0.6)
    f.write(urls) #用encode('utf-8)    #最後再一次寫出來
    f.close()    

http://www.appledaily.com.tw/appledaily/archive/20160312


## 步驟四、讀取一天的url檔案，把檔案抓下來

In [154]:
#結合新聞類別
import string
import datetime as dt
import requests
import time
from bs4 import BeautifulSoup
import json
import ast  #轉換成json需要套件
import time
count = 0
L = []
#製造日期的迴圈
startdate = dt.datetime(2016, 3,12)
endate = dt.datetime(2016, 3,12)
totaldays = (endate - startdate).days + 1
all_json_str = ''
for daynumber in range(totaldays):
    datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y%m%d") 
    #開起每天的url檔案
    f = open('C:/Users/BIG DATA/Desktop/Apple_news/{}.text'.format(datestring),'r')
    for url_no in f.readlines():
        try:
            category = url_no.split('http')[0]        #把類別抓出來
            url = 'http'+url_no.split('http')[1]      #重新組合url
            res = requests.get(url)
            soup = BeautifulSoup(res.text)
            file_date,file_num = url.split('/')[6],url.split('/')[7]  #替檔案取名子  
            f = open('C:/Users/BIG DATA/Desktop/Apple_news/{}_{}.text'.format(file_date,file_num),'w')  

            #抓標題跟副標題
            title1 = soup.select('#h1')[0].text
            title2 = soup.select('#h2')[0].text            
            title =''
            titles = str(title1.encode('utf-8'))+str(title2.encode('utf-8'))
            title = ','.join(titles.split())
            #print title
            
            #抓內文
            article = ''
            div= soup.select('.articulum')[0]   #先把所有內文抓到
            ps = div.select('p')                #再把裡面所有的p標籤抓到
            for p in ps:                        #把每個p標籤一個一個抓出來印
                article += p.text.replace('\r','')  #段落會隱藏換行符號\r，要取代掉，不然後面讀取json格式會只讀到最後一行

            #抓日期
            #date = soup.select('.gggs')[0].select('time')[0].text
            #f.write(time.encode('utf-8')+'\n')
        
            data  =  { 
                'comp':'apple',  #新聞來源
                'category':category,
                'title':title, 
                'content':article,
                'date':file_date,  #直接抓檔案的日期來用
                'url':url
            }
             
            all_json_str += str(data)+','  #加個,把多本字典分開來
            #內文抓取檔案關閉
            time.sleep(0.6)
            f.close()
    
        #印出有問題的新聞
        except IndexError:
            count +=1
            print count
            continue
    #url 檔案關閉
    f.close()
    
f1 = open('C:/Users/BIG DATA/Desktop/Apple_news/json/{}.json'.format(datestring),'w')
#為了讓多本字典存到同一個檔案之中，會在外面產生[]
a = eval(all_json_str) #AST套件 把多本字典轉換成 list (eval是可剝離最外層包裝的函式)
#----------------------------------------------
b =json.dumps(a) #再把 list 字典轉換成 json 並包起來
f1.write(b)
f1.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
<type 'str'>


In [159]:
print type(all_json_str)
print type(a)
print type(b)

<type 'str'>
<type 'list'>


NameError: name 'b' is not defined

## 讀取json檔

In [26]:
#讀取json檔
import json
with open('C:/Users/BIG DATA/Desktop/Apple_news/json/201304.json', 'r') as f:
    news = json.load(f) #整個檔案屬於一個物件，每篇新聞用list的方式存著，因此要一篇一篇讀出來
    print type(news)
for i in range(0,len(news)):
    print news[i]['title'],news[i]['date']

<type 'list'>
真相浮出,貪4千萬珠寶奪2命蛇蠍女曬鑽戒,惹怒人妻 20130401
8死亡拼圖,拆魔女謊言死者15撮髮,纏水筆仔林,冥冥帶路 20130401
殺人怕男友變心,女拖老闆當墊背 20130401
媽媽嘴將重開張,網友：願再喝 20130401
人染H7N9全球首見,中國禽流感2死台入境警戒,發燒38℃,篩檢後送醫 20130401
雙子星案再查4議員,恐約談郝龍斌 20130401
澳洲打工遊學展4/20,4/21華山登場 20130401
收押首日胃口差,賴素如坐床沉思 20130401
婚姻觸礁,于美人：還想挽回夫家控媳大小聲,「家門不幸」 20130401
「想天長地久」,吳淡如勤侍公婆 20130401
徵求原味絲襪,「臭到炸,加錢」變態誆試穿,8女大生受害 20130401
威力彩今衝4.9億,「0224」熱門 20130401
蘋論：誰才是賤人 20130401
司馬觀點：朝鮮核武夢（江春男） 20130401
國中會考分等更細,政策轉彎「把人當笨蛋」 20130401
數學考非選題,師：比基測難 20130401
砸億元,缺考3成,教部竟滿意 20130401
桂林翻船,台遊客1失蹤與妻同遊遇暴雨,船工操作不當 20130401
考題難,學生描愛心惡搞答案卡 20130401
賣麵撐家,11歲女夜市人生23自強童,困境中堅毅向上 20130401
國三女苦讀,曬藥草擔家計 20130401
李朝卿若復職,「民眾圍縣府」施明德嗆馬：不能叫李放長假嗎 20130401
《蘋果數》英國樂透開獎,44最夯出現248次 20130401
北部,今日提醒你 20130401
中部,今日提醒你 20130401
南部,今日提醒你 20130401
我很不爽 20130401
打房政策發酵,低於實價登錄房屋最搶手329檔期待售物件激增,買家關注低價宅,合宜宅周邊區域成點閱熱門 20130401
模範警,襲胸同僚國二女「披羊皮的狼」,僅調動處分,民控「私了」 20130401
飆車族猝死,fb召黑衣人送葬恐街頭鬧事,警攔查29青少年 20130401
酒駕翻車,男抱女友怨路爛 20130401
國道攔車,惡煞逼停踹駕駛 20130401
冒辦手機牟利,300客戶受害 20130401
洗衣精,僅85元 20130401
降價2成,Canon相